<a href="https://colab.research.google.com/github/tjdrhks0808/Seminar_in_Statistics/blob/gh-pages/NLP_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%95%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm
import re

In [2]:
xlsx = pd.read_excel("/content/drive/MyDrive/NLP/공모전_제공_데이터(1차).xlsx")

In [3]:
data = xlsx['발화']

In [4]:
length = len(data)

# 구두점/공백 제거

In [ ]:
only_word = []
for i in tqdm(range(length)):
  tmp = re.compile('[^가-힣a-zA-Z0-9 | ㄱ-ㅎ | ㅏ-ㅣ]').sub("",data[i])
  only_word.append(tmp)

In [ ]:
no_space = []
for i in tqdm(range(length)):
  tmp = re.sub(r"\s+", "", only_word[i])
  no_space.append(tmp)

#자동 띄어쓰기

In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import Spacing

#Test
spacing = Spacing()
kospacing_data = spacing(no_space[0]) 

print(no_space[0])
print(kospacing_data)

상담직원과빠른연결했슴좋겠어요
상담직원과 빠른 연결했슴 좋겠어요


In [ ]:
spc_data = []

for i in tqdm(range(length)):
  spacing = Spacing()
  tmp = spacing(no_space[i]) 
  spc_data.append(tmp)

100%|██████████| 1000/1000 [01:05<00:00, 15.20it/s]


# 맞춤법 검사

In [7]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-cyaiqcew
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-cyaiqcew
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=d06c5249fabf95e16f25516fa4a91eee4b2d62a165d96adf3ad29f37e79d7c17
  Stored in directory: /tmp/pip-ephem-wheel-cache-568_5r_r/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [8]:
from hanspell import spell_checker

#Test
spelled_sent = spell_checker.check(data[0])
hanspell_sent = spelled_sent.checked
print(data[0])
print(hanspell_sent)

상담직원과 빠른연결했슴좋겠어요
상담직원과 빠른연결했슴좋겠어요


In [10]:
spell_data = []

for i in tqdm(range(length)):
  tmp = spell_checker.check(data[i])
  hanspell_tmp = tmp.checked
  spell_data.append(hanspell_tmp)

 32%|███▏      | 318/1000 [01:11<03:08,  3.62it/s]

ParseError: ignored

# 셀레니움 활용 부산대 맞춤법

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
def spell_checking(sentence):
  time.sleep(0.5)
  wd.get('http://speller.cs.pusan.ac.kr/')
  try:
    wd.find_element_by_xpath('//*[@id="text1"]').send_keys(sentence)
    wd.find_element_by_xpath('//*[@id="btnCheck"]').click()
    time.sleep(1)
    entity_num = 0
    while True:
      try:
        wd.find_element_by_xpath('//*[@id="tdReplaceWord_'+str(entity_num)+'"]/ul/li/a').click()
        entity_num += 1
      except:
        break
    texts = wd.find_element_by_xpath('//*[@id="tdCorrection1stBox"]').text
    return texts
  except:
    return sentence

In [ ]:
spell_data2 = []

for i in tqdm(range(length)):
  tmp = spell_checking(spc_data[i])
  spell_data2.append(tmp)

100%|██████████| 1000/1000 [39:18<00:00,  2.36s/it]


# 원본과 대조

In [ ]:
compare = pd.DataFrame()
compare['원본'] = data
compare['네이버'] = spell_data
compare['부산대'] = spell_data2

In [ ]:
compare[300:320]

,원본,네이버,부산대
300,다른제유보험회사전화불미스럽습니다,다른 제 뉴 보험회사 전화 불미스럽습니다,다른 제유보험회사 전화 불미스럽습니다
301,대출관련 마케팅 문자 권유 불쾌함,대출 관련 마케팅 문자 권유 불쾌함,대출 관련 마케팅 문자 권유 불쾌함
302,광고제발 놉,광고제 발놉,광고제 발본
303,쓸데없이ㅡ전화하지ㅡ않았으면 좋겠습니다.,쓸데없이 ㅡ 전화하지ㅡ 않았으면 좋겠습니다,쓸데없이 전화하지 않았으면 좋겠습니다
304,전화가너무많이걸 려옵니다,전화가 너무 많이 걸려옵니다,전화가 너무 많이 걸려옵니다
305,문자나 메일로 상품 광고 안내를 했으면.좋을듯...근무중 상품 광고를 전화를 받...,문자나 메일로 상품 광고 안내를 했으면 좋을 듯 근무 중 상품 광고를 전화를 받으면...,문자나 메일로 상품 광고 안내를 했으면 좋을 듯 근무 중 상품 광고를 전화를 받으면...
306,제휴 보험회사 연락이 안왔으면 해요,제휴 보험회사 연락이 안 왔으면 해요,제휴보험회사 연락이 안 왔으면 해요
307,마케팅 활용 전화시 미리 보험이면 보험이라고 얘기를 하고 통화 지속여부 확인하고 통...,마케팅 활용 전화 시 미리 보험이면 보험이라고 얘기를 하고 통화 지속 여부 확인하고...,마케팅 활용 전화 시 미리 보험이면 보험이라고 얘기를 하고 통화 지속 여부 확인하고...
308,삼성카드는 그렇지 않겠지만 타 카드 회사는 보험회사와 연결되는 전화가 오는 경우가 ...,삼성카드는 그렇지 않겠지만 타 카드 회사는 보험회사와 연결되는 전화가 오는 경우가 ...,삼성카드는 그렇지 않겠지만 타 카드 회사는 보험회사와 연결되는 전화가 오는 경우가 ...
309,연계된 제휴사 (보험)안내 없는카드사되주세요,연계된 제휴사 보험 안내 없는 카드사 돼주세요,연계된 제휴사 보험 안내 없는 카드사되 주세요


In [ ]:
compare.to_csv('/content/drive/MyDrive/NLP/데이터_정제.csv')